In [1]:
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup

import time


In [2]:
firefox_service = Service()
web_driver = webdriver.Firefox(service = firefox_service)

In [3]:
url = "https://www.scopus.com/authid/detail.uri?authorId=57206520256"
web_driver.get(url)

In [10]:
def get_author_data(web_driver, scopus_id):
	author_soup = BeautifulSoup(web_driver.page_source,'lxml')
	web_driver.find_element(By.ID, "AuthorHeader__showAllAuthorInfo").click() #Abrir aba de dados
	time.sleep(1)
	sidebar_soup = BeautifulSoup(web_driver.page_source, 'lxml')
	web_driver.find_element(By.CSS_SELECTOR, "button[data-test-id='flyout-close-button']").click() #Fecha aba de dados

	author_information = {"ID": scopus_id}

	name = author_soup.find("title").string[:-26]
	author_information["Name"] = name

	author_location = author_soup.find("span", {"data-testid": "authorInstitution"})
	found_elements = author_location.find_all("span")
	location = found_elements[1].string[2:].split(sep = ",")
	author_information["Country"] = location[1].strip()
	author_information["City"] = location[0]
	author_information["Institution"] = found_elements[0].string

	found_elements = author_soup.find_all("span", {"data-testid": "unclickable-count"})
	author_information["Citations"] = found_elements[0].string
	author_information["Documents"] = found_elements[1].string
	author_information["h-index"] = found_elements[2].string

	author_areas = sidebar_soup.find("span", {"data-testid": "authorSubjects"}).string.split(sep = " • ")
	author_information["Areas"] = author_areas

	return author_information


In [11]:
id = url[url.find("=")+1:]
print(id)

get_author_data(web_driver, id)


57206520256


{'ID': '57206520256',
 'Name': 'Colditz, Graham A.',
 'Country': 'United States',
 'City': 'St. Louis',
 'Institution': 'Washington University School of Medicine in St. Louis',
 'Citations': '220,451',
 'Documents': '1,486',
 'h-index': '246',
 'Areas': ['Medicine',
  'Biochemistry, Genetics and Molecular Biology',
  'Agricultural and Biological Sciences',
  'Social Sciences',
  'Nursing',
  'Mathematics',
  'Psychology',
  'Decision Sciences',
  'Health Professions',
  'Neuroscience',
  'Immunology and Microbiology',
  'Pharmacology, Toxicology and Pharmaceutics',
  'Dentistry',
  'Arts and Humanities',
  'Environmental Science',
  'Engineering',
  'Multidisciplinary',
  'Chemical Engineering',
  'Economics, Econometrics and Finance',
  'Materials Science',
  'Chemistry',
  'Physics and Astronomy']}

In [12]:
web_driver.quit()